# RADGA

In [1]:
#| default_exp 53-encoder-parallel-radga-with-cross-attention-loss-component-for-wikititles

In [2]:
%load_ext autoreload
%autoreload 2

from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [3]:
#| export
import os,sys,torch,pickle,torch.multiprocessing as mp, pickle
from xcai.basics import *
from xcai.models.radga import RAD001

In [4]:
os.environ['WANDB_MODE'] = 'disabled'

In [5]:
#| export
os.environ['CUDA_VISIBLE_DEVICES'] = '8,9,10,11,12,13'
os.environ['WANDB_PROJECT']='xc-nlg_53-encoder-parallel-radga-with-cross-attention-loss-component-for-wikititles'

## Data

In [6]:
#| export
data_dir = '/home/aiscuser/scratch/datasets'

In [7]:
block = XCBlock.from_cfg(data_dir, 'data_metas', dset='wikititles', valid_pct=0.001, tfm='rm', tokenizer='distilbert-base-uncased', 
                         smp_features=[('lbl2data',1, 1), ('sal2data',1, 1), ('hlk2data',1,3)], 
                         n_data_meta_samples=50, n_lbl_meta_samples=50)

/home/aiscuser/.local/lib/python3.9/site-packages/xclib-0.97-py3.9-linux-x86_64.egg/xclib/data/data_utils.py:263: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


In [8]:
#| export
pkl_dir = f'{data_dir}/processed/'

In [9]:
with open(f'{pkl_dir}/wikititles_data-metas_distilbert-base-uncased_rm_radga-final.pkl', 'wb') as file: 
    pickle.dump(block, file)

In [8]:
#| export
with open(f'{pkl_dir}/wikititles_data-metas_distilbert-base-uncased_rm_radga-final.pkl', 'rb') as file: 
    block = pickle.load(file)

In [10]:
#| export
args = XCLearningArguments(
    output_dir='/home/aiscuser/outputs/53-encoder-parallel-radga-with-cross-attention-loss-component-for-wikititles-1-0',
    logging_first_step=True,
    per_device_train_batch_size=200,
    per_device_eval_batch_size=100,
    representation_num_beams=200,
    representation_accumulation_steps=1,
    save_strategy="steps",
    evaluation_strategy='steps',
    eval_steps=1000,
    save_steps=1000,
    save_total_limit=5,
    num_train_epochs=300,
    predict_with_representation=True,
    adam_epsilon=1e-6,
    warmup_steps=100,
    weight_decay=0.01,
    learning_rate=2e-4,
    generation_num_beams=10,
    generation_length_penalty=1.5,
    predict_with_generation=True,
    representation_search_type='INDEX',
    group_by_cluster=True,
    num_clustering_warmup_epochs=10,
    num_cluster_update_epochs=5,
    num_cluster_size_update_epochs=10,
    clustering_type='EXPO',
    minimum_cluster_size=1,
    maximum_cluster_size=300,
    output_concatenation_weight=1.0,
    use_encoder_parallel=True,
    metric_for_best_model='P@1_REPR',
    target_indices_key='plbl2data_idx',
    target_pointer_key='plbl2data_data2ptr',
    fp16=True,
    label_names=['sal2data_idx', 'sal2data_input_ids', 'sal2data_attention_mask',
                 'hlk2data_idx', 'hlk2data_input_ids', 'hlk2data_attention_mask'],
)

/opt/conda/envs/ptca/lib/python3.9/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
#| export
test_dset = block.test.dset.sample(n=2000, seed=50)
metric = PrecRecl(block.n_lbl, test_dset.data.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])

In [12]:
#| export
bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()


model = RAD001.from_pretrained('distilbert-base-uncased', num_batch_labels=5000, ignore_token=0, batch_size=bsz,
                               margin=0.3, num_negatives=5, tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix='hlk2data', lbl2data_aug_meta_prefix='hlk2lbl', 
                               resize_length=5000,
                               
                               gen_loss_weight=0.001, meta_loss_weight=0.3, pred_meta_prefix='sal', 
                               
                               fusion_loss_weight=0.1, tie_word_embeddings=False,
                               
                               use_fusion_loss=True, use_noise=True, use_encoder_parallel=True)

model.init_retrieval_head()
model.init_generation_head()

Some weights of RAD001 were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.layer_norm.bias', 'encoder.cross_head.layer_norm.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight', 'encoder.gen_head.projector.weight', 'encoder.meta_head.layer_norm.bias', 'encoder.meta_head.layer_norm.weight', 'encoder.meta_head.projector.bias', 'encoder.meta_head.projector.weight', 'encoder.meta_head.transform.bias', 'encoder.meta_head.transform.weight', 'vocab_projector.weight']
You should probably TRAIN this model on a down-stream task

In [13]:
#| export
trie = XCTrie.from_block(block)

  0%|          | 0/501070 [00:00<?, ?it/s]

In [14]:
#| export
learn = XCLearner(
    model=model, 
    args=args,
    trie=trie,
    train_dataset=block.train.dset,
    eval_dataset=test_dset,
    data_collator=block.collator,
    compute_metrics=metric,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
learn.train()

In [ ]:
#| export
if __name__ == '__main__':
    mp.freeze_support()
    learn.train()
    